In [30]:
import tensorflow as tf
import math
import numpy as np
import matplotlib.pyplot as plt

In [31]:
IMAGE_SIZE = 256
BATCH_SIZE = 32

In [32]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'Pistachio_Image_Dataset\Pistachio_Image_Dataset',
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
    
)

Found 2148 files belonging to 2 classes.


In [33]:
len(dataset.class_names)

2

In [34]:
number_ds = len(dataset) * BATCH_SIZE
print(number_ds)

2176


In [35]:
def split_data(dataset, train_split = 0.8, val_split = 0.1 , test_split = 0.1 , shuffle = True , shuffle_size = 1000): 
        dataset_size  = len(dataset)
        train_size = int(train_split* dataset_size)
        val_size = int(val_split * dataset_size)
        test_size = int(test_split * dataset_size)
        
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size).skip(val_size)
        if shuffle == True:
            dataset = dataset.shuffle(shuffle_size)
        return  train_ds, val_ds, test_ds 

In [36]:
train_ds , val_ds , test_ds =split_data(dataset)

In [37]:
len(train_ds)

54

In [38]:
len(test_ds)

8

In [39]:
len(val_ds)

6

In [40]:
train_ds.cache().prefetch(buffer_size = tf.data.AUTOTUNE).shuffle(1000)
test_ds.cache().prefetch(buffer_size = tf.data.AUTOTUNE).shuffle(1000)
val_ds.cache().prefetch(buffer_size = tf.data.AUTOTUNE).shuffle(1000)

<ShuffleDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [41]:
model= tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255),
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.Conv2D(32,(3,3), activation = 'relu' , input_shape = (IMAGE_SIZE,IMAGE_SIZE)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(2, activation = 'softmax')
])

In [42]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
epochs = 30
history = model.fit(
    train_ds,
    epochs=epochs,
    verbose = 1,
    validation_data = val_ds
)

Epoch 1/30


In [ ]:
score = model.evaluate(test_ds)

In [ ]:
history.history.keys()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()